In [0]:
!pip install pydrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

from matplotlib import pyplot as plt
import pandas as pd
from copy import deepcopy
import numpy as np

def GoogleDriveAuth(ID):
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file = drive.CreateFile({'id': ID})
    return file

File_ID = '1XYOhPmDIQNexg-XgOgP29qc1KxVleXwt'
file_name = GoogleDriveAuth(File_ID)
file_name.GetContentFile('training.1600000.processed.noemoticon.csv')
train_data = pd.read_csv('training.1600000.processed.noemoticon.csv',encoding='latin-1',header=None)
train_data = pd.DataFrame(train_data)

File_ID = '1VHDAjvUvaxwI-VoGbOdGX5RHqxsadQTP'
file_name = GoogleDriveAuth(File_ID)
file_name.GetContentFile('testdata.manual.2009.06.14.csv')
test_data = pd.read_csv('testdata.manual.2009.06.14.csv',header=None)
test_data = pd.DataFrame(test_data)

In [0]:
from sklearn.utils import shuffle
train_data=shuffle(train_data[[0,5]]).reset_index(drop=True)
test_data=shuffle(test_data[[0,5]]).reset_index(drop=True)
print(train_data.head())
print(test_data.head())
print(train_data[0].value_counts())
print(test_data[0].value_counts())

test_data=test_data.loc[test_data[0]!=2]
print(test_data[0].value_counts())

   0                                                  5
0  0                             tired  goodnight    L 
1  0   well you didn t ask me if i wanted to move wi...
2  0  is very sad    get well soon my beloved Cik Bi...
3  4                                happy mother s day 
4  4     watching Canadian Nationals then JMO s Prom   
   0                                                  5
0  2  all about Ajax,jquery ,css ,JavaScript and mor...
1  2           Going to see star trek soon with my dad.
2  2  How to implement a news ticker with jQuery and...
3  0  Give a man a fish, u feed him for the day. Tea...
4  4  @sklososky Thanks so much!!! ...from one of yo...
0    510
4    490
Name: 0, dtype: int64
4    182
0    177
2    139
Name: 0, dtype: int64
4    182
0    177
Name: 0, dtype: int64


In [0]:
training_examples=10000
train_data[0].value_counts()
train_data=train_data[:training_examples]
train_data[0].value_counts()

In [0]:
#Clean
import re
from bs4 import BeautifulSoup
for i in range(training_examples):
  train_data[5][i] = BeautifulSoup(train_data[5][i], 'lxml').get_text()   #HTML decoding
  train_data[5][i]=re.sub(r'@[A-Za-z0-9]+','',train_data[5][i])           #@______
  train_data[5][i]=re.sub('https?://[A-Za-z0-9./]+','',train_data[5][i])  #url
  train_data[5][i]=re.sub("[^a-zA-Z]", " ", train_data[5][i])
  train_data[5][i]=train_data[5][i].encode('utf-8').decode('utf-8').replace(u"\ufffd", "?") #Byte-Order-Mark
  



In [0]:
print(train_data.shape)
train_data=train_data[:1000]
train_data[5]

(1000, 2)


0      Photo  maichau  b  a      p       p nh       T...
1       OOOOH excellent  I WILL be listening and figh...
2       yeah i loved the books  i couldnt even finish...
3      Placed  th in funrun today in the       age gr...
4      it s nice what sony has shown  but no price dr...
5       I have sent you instructions for loading pics...
6       No worries  Sharing the good news about our m...
7       family is crazyyyy   lol grr ima go and wait ...
8      i found all my gameboy games   disappointed i ...
9       But SSIS is like an old friend and I hate reg...
10     ARGH      what the hell is going on with the d...
11     I have more gallbladder induced vomiting befor...
12                                          thank you  x
13     ALLi ThAt GiRL has free TACO BELL for   people...
14                                  Thank you  my dear  
15      lol  Man  I woke up thinkin about all kinds o...
16     I m definitely excited to go back home to Gran...
17     Feels like a act scratch

In [0]:

import scipy
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data[5]).toarray()    # 01 2*2 matrix
X_train = pd.DataFrame.from_records(X_train, columns=vectorizer.get_feature_names())

#print(ktest)
X_test=vectorizer.fit_transform(test_data[5][:10]).toarray() 
X_test = pd.DataFrame.from_records(X_test, columns=vectorizer.get_feature_names())    #makes column of different words
print(X_test.shape)


#print(X.shape)
# print(X_test.shape)
# print(Y_train.shape)
# print(Y_test.shape)
X_train
X_train.shape

(10, 107)


(1000, 3127)

In [0]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial').fit(X_train, train_data[0])


X_test=X_test.append(X_train)    #merge two lists
X_test=X_test[X_train.columns]  #keep only X_train columns

where_are_NaNs = np.isnan(X_test)    #find new columns
X_test[where_are_NaNs] = 0    #fill Nan with 0 an not present

print('Predicted')
print(clf.predict(X_test[0:100]))
print('Actual')
print(((test_data[0][:100]).as_matrix()))
print('Accuracy=',clf.score(X_test[0:100], test_data[0][:100]))

Predicted
[4 4 0 4 4 4 4 0 4 4 0 0 0 4 4 0 0 4 4 0 0 4 4 0 4 0 4 4 4 0 4 0 4 0 4 0 4
 0 4 4 0 4 0 4 4 4 0 0 0 0 0 4 4 0 4 0 4 0 4 0 4 4 0 4 4 4 0 4 4 0 0 0 0 4
 0 4 4 4 4 0 4 0 4 0 4 0 4 0 0 0 4 0 4 0 0 4 0 4 4 4]
Actual
[0 4 0 0 0 0 4 4 4 4 4 0 0 0 0 0 0 0 0 0 0 4 0 4 4 4 0 0 4 0 4 4 4 0 4 4 4
 0 0 0 4 4 4 4 4 4 4 4 0 0 4 4 4 0 0 0 0 0 0 4 4 4 0 0 4 0 4 0 4 0 0 0 0 0
 0 0 4 0 0 4 4 4 4 0 0 0 0 4 0 0 4 4 0 4 0 4 4 4 4 4]
Accuracy= 0.55
